In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [ ]:
#Used pandas to read csv file
df = pd.read_csv('damped_harmonic_oscillator_data.csv')

t = df['t'].to_numpy()
x = df['x'].to_numpy()
print(df)
plt.plot(t,x)
plt.xlabel('Time')
plt.ylabel('Position')
plt.title('Given data')
plt.show()

In [ ]:
T = t.reshape(-1, 1)  # time
X = x.reshape(-1, 1)  # position
print(T[0],X[0])
print(T.dtype)

In [ ]:
#splitting data [40 % testing and 60% training]
#20%(test) ---- 60% (train)--- 20% (test)

marker_1 = int(0.2 * len(df))
marker_2 = int(0.8 * len(df))

# TRAINING DATA
train_data = df.iloc[marker_1:marker_2]
T_train = train_data.iloc[:, :1].to_numpy().reshape(-1, 1)  # time
X_train = train_data.iloc[:, 1:].to_numpy().reshape(-1, 1)  # position

# TESTING DATA
first_half_test = df.iloc[:marker_1]
second_half_test = df.iloc[marker_2:]

test_data = pd.concat([first_half_test, second_half_test])

T_test = test_data.iloc[:, :1].to_numpy().reshape(-1, 1)  # time
X_test = test_data.iloc[:, 1:].to_numpy().reshape(-1, 1)  # position

# Verify shapes
print("Shape of T_train:", T_train.shape)
print("Shape of X_train:", X_train.shape)
print("Shape of T_test:", T_test.shape)
print("Shape of X_test:", X_test.shape)

In [ ]:
# Convert numpy arrays to PyTorch tensors
T_train_tensor = torch.tensor(T_train, dtype=torch.float32)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
T_test_tensor = torch.tensor(T_test, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

In [ ]:
torch.seed()

class SGDRegression(nn.Module):
    def __init__(self, activation_fn):
        super(SGDRegression, self).__init__()
        self.fc1 = nn.Linear(1, 16)  # Input layer [1 input]
        self.fc2 = nn.Linear(16, 16)  # Hidden layer 1
        self.fc3 = nn.Linear(16, 16)  # Hidden layer 2
        self.fc4 = nn.Linear(16, 1)   # Output layer [1 output]
        self.activation_fn = activation_fn

    def forward(self, x):
        x = self.activation_fn(self.fc1(x))  
        x = self.activation_fn(self.fc2(x)) 
        x = self.activation_fn(self.fc3(x))  
        x = self.fc4(x)
        return x

In [ ]:
activation_functions = {
    'ReLU': F.relu,
    'Sigmoid': torch.sigmoid,
    'Tanh': torch.tanh,
    'LeakyReLU': F.leaky_relu,
    'ELU': F.elu,
    'SELU': F.selu,
    'GELU': F.gelu,
    'Softplus': F.softplus,
    'Softsign': F.softsign,
    'ReLU6': F.relu6,
    'Hardtanh': F.hardtanh,
    'LogSigmoid': F.logsigmoid
}
# Dictionary to store losses corresponding to each actvation function
all_losses = {name: [] for name in activation_functions}

In [ ]:
BATCH_SIZE = 200

train_data = torch.utils.data.TensorDataset(T_train_tensor, X_train_tensor)
data_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

for activation_name, activation_fn in activation_functions.items():
    model = SGDRegression(activation_fn)
    loss_fn = torch.nn.MSELoss() # we want to model mean square loss
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    EPOCHS = 50
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    losses = []
    
    for epoch in range(EPOCHS):
        for batch, (input_data, output_data) in enumerate(data_loader):

            X_pred = model(input_data)
            loss = loss_fn(X_pred, output_data)
            losses.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
    
    all_losses[activation_name] = losses

In [ ]:
# Plotting the loss curves in subplots
num_activations = len(activation_functions)
cols = 4  
rows = (num_activations + cols - 1) // cols 

fig, axes = plt.subplots(rows, cols, figsize=(20, 4 * rows))
axes = axes.flatten()

for i, (activation_name, losses) in enumerate(all_losses.items()):
    ax = axes[i]
    ax.plot(losses)
    ax.set_title(activation_name)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')

plt.tight_layout()
plt.show()

In [ ]:
#Evaluating
model.eval()
with torch.no_grad():
    X_test_pred = model(T_test_tensor)
    X_train_pred = model(T_train_tensor)
X_test_pred = X_test_pred.numpy()
X_train_pred = X_train_pred.numpy()

# Plot actual vs predicted values
plt.plot(T_test, X_test, 'g.', label='Test Actual')
plt.plot(T_test, X_test_pred,'r.', label='Test Predicted')
plt.plot(T_train, X_train, 'y.', label='Train Actual')
plt.plot(T_train, X_train_pred,'b.', label='Train Calculated')

plt.xlabel('Time (t)')
plt.ylabel('Position (x)')
plt.legend()
plt.title('Actual vs Predicted Values')
plt.legend()
plt.show()